In [23]:
import subprocess
import json


def derive_wallets(coin):

    command = 'php derive -g --mnemonic="wish mistake trumpet title tonight desert web cannon tumble pause lemon prepare" --cols=path,address,privkey,pubkey --format=json'
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, error) = p.communicate()
    
    p_status = p.wait()
    
    keys =json.loads(output)
    
    return (keys)

In [24]:
from web3 import Web3
from web3.middleware import geth_poa_middleware

from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

In [25]:
from constant import *


In [26]:
from dotenv import load_dotenv
load_dotenv()
mnemonic = os.getenv('mnemonic', 'wish mistake trumpet title tonight desert web cannon tumble pause lemon prepare')

In [32]:
coins = {
    'btc-test' : derive_wallets(BTCTEST), 
    'eth': derive_wallets(ETH)
 }

print(coins)

{'btc-test': [{'path': "m/44'/0'/0'/0/0", 'address': '1MgahxFtoKbWbHgFt9DPYaBv3W4kNFGsDG', 'privkey': 'L1JLVNmiyhUpRp3qRxbaobgGuiTVc3aT8Q8o1LSfP2y5BBdceZy4', 'pubkey': '03ed1af31a5a284cc88ebe30130182dfe058b7d4a94734768d1354f1aea0c01e0b'}, {'path': "m/44'/0'/0'/0/1", 'address': '1GiepSWoFkMHaVVUPz5bn68g6hxL1JptyR', 'privkey': 'L2VY2LJoXsNhSpBXJmdm9MTo7NAUW4bsZVVxt7UymczM6WEomZHG', 'pubkey': '033569a979b8fec2e8dfb6f0eecaf02003b9eb674a3061ba45282107afe62d987c'}, {'path': "m/44'/0'/0'/0/2", 'address': '15oS4hstUTfVbXzUa6BTicG1wfHchbbXaL', 'privkey': 'KxmaauYmbLM97VvFeXqS3aKBPej6kSXBKZnmQv2X9gN958zo5d6P', 'pubkey': '0379ab2e6a934a2f393a55be54d4e1be3b4b7bdabbeffc6c34dc0ced37ae94da9e'}, {'path': "m/44'/0'/0'/0/3", 'address': '1Mfq82rZyAkwTNBaGSTXQiPi2Una1j9YC7', 'privkey': 'Kx5ZCZ7MdfgcsH8cq5mS48pS5UFFANnmf1Wq5DoNxJ3WNaZr8e7w', 'pubkey': '02510a953f6fb48e7b2fc9855ed722e7c3da448e1ea7230f41aa7edf79c4351e51'}, {'path': "m/44'/0'/0'/0/4", 'address': '189cPjV4W6SpmvG2fPe4Cg96esBTaNo2FS', 'privkey'

In [33]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

In [34]:
def priv_key_to_account(coin,priv_key): 

    if coin==BTCTEST:
        return PrivateKeyTestnet(priv_key)
    elif coin==ETH:
        return Account.privateKeyToAccount(priv_key)

In [45]:
def create_tx(coin, account, to, amount): 
    if coin==BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    elif coin==ETH:
        gas_estimate = w3.eth.estimateGas(
            {'from': account.address, 'to': to,'value': amount}
        )
        return {
            'from' : account.address,
            'to': to,
            'value': amount,
            'gasPrice': w3.eth.gasPrice,
            'gas': gas_estimate,
            'nonce': w3.eth.getTransactionCount(account.address)
        }


In [46]:
def send_tx(coin, account, to, amount):
    raw_tx = create_tx(coin,account,to,amount)
    signed = account.sign_transaction(raw_tx)
    if coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed)
        
    elif coin==ETH:
        return w3.eth.sendRawTransaction(signed.rawTransaction)

In [49]:
private_key = "0xccb16662cd5c1a83d0806300aeede26ef190060fc4e8deb315289c421fc4ab8b"

In [50]:
my_account = Account.from_key(private_key)
print(my_account.address)

0xAa4E22b9cE31424dAde4D1023b0baeE81Db10d37


In [61]:
from web3.middleware import geth_poa_middleware

In [62]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)